# Solution Explanation

I classiifed the Swedish leaf dataset into leaves belong to Ulmus trees (Ulmus carpinifolia (folder no.1 in the dataset) and Ulmus glabra (folder no. 9 in the dataset)) and leaves that don't (all the other trees in the dataset).
For the model I used a pre-trained VGG16 model and added layers on top of it for binary classification.

#    Data Loading and preprocessing

In [1]:
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import os
import cv2
import numpy as np

In [2]:
def load_leaves_from_folder(foldername):
    images = []
    for filename in os.listdir(foldername):
        img = cv2.imread(os.path.join(foldername,filename))
        if img is not None:
            img = cv2.resize(img,(224,224))
            images.append(img)
    return images

In [3]:
# load the data and set their labels - 1 for ulmus leaves, 0 to non ulmus

data =[]
for leaf in load_leaves_from_folder("ulmus_leaves"):
    data.append([leaf,1])
for leaf in load_leaves_from_folder("not_ulmus_leaves"):
    data.append([leaf,0])

In [4]:
x = []
y = []
for leaf in data:
    x.append(leaf[0])
    y.append(leaf[1])

x = np.array(x)
y = np.array(y)   

In [5]:
from sklearn.model_selection import train_test_split

# split the data to train and test
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

# Model Building and Training

The pre-trained model which I used is a VGG16 model trained on the imagenet dataset. 
The last output layer is removed, and the layers are set to be untrainable in order to keep the original weights

In [6]:
from tensorflow.keras.applications.vgg16 import VGG16

base_model = VGG16(input_shape = (224, 224, 3), include_top = False, weights = 'imagenet')

for layer in base_model.layers:
    layer.trainable = False

In [7]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

On top of the pre-trained VGG16 I added a fully connected (dense) layer, then a dropout of 0.5 to prevent overfitting and an output dense layer with a singal output activated by the sigmoid function for binary classification. 
The model is optimized via Adam optimizer and a binary crossentropy loss.

In [8]:
x = layers.Flatten()(base_model.output)

x = layers.Dense(512, activation='relu')(x)

x = layers.Dropout(0.5)(x)

x = layers.Dense(1, activation='sigmoid')(x)

model = Model(base_model.input, x)

model.compile(optimizer = Adam(lr=0.0001), loss = 'binary_crossentropy',metrics = ['accuracy'])

C:\Users\talso\anaconda3\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [9]:
mode_train = model.fit(X_train,y_train,epochs=10,batch_size=50)

Epoch 1/10
16/16 [==============================] - 139s 9s/step - loss: 1.3168 - accuracy: 0.9072
Epoch 2/10
16/16 [==============================] - 152s 9s/step - loss: 0.3714 - accuracy: 0.9708
Epoch 3/10
16/16 [==============================] - 149s 9s/step - loss: 0.1691 - accuracy: 0.9873
Epoch 4/10
16/16 [==============================] - 153s 10s/step - loss: 0.0609 - accuracy: 0.9936
Epoch 5/10
16/16 [==============================] - 158s 10s/step - loss: 0.0460 - accuracy: 0.9962
Epoch 6/10
16/16 [==============================] - 161s 10s/step - loss: 0.0085 - accuracy: 0.9975
Epoch 7/10
16/16 [==============================] - 156s 10s/step - loss: 0.0267 - accuracy: 0.9962
Epoch 8/10
16/16 [==============================] - 159s 10s/step - loss: 0.0093 - accuracy: 0.9987
Epoch 9/10
16/16 [==============================] - 164s 10s/step - loss: 0.0367 - accuracy: 0.9924
Epoch 10/10
16/16 [==============================] - 167s 10s/step - loss: 0.0890 - accuracy: 0.9911


# Prediction and Evaluation

In [10]:
preds = model.predict(X_test)

In [11]:
# threshold the output values to zeros and ones for evaluation

preds_binary =[]
for pred in preds:
    if pred > 0.5:
        preds_binary.append(1)
    else:
        preds_binary.append(0)

In [12]:
from sklearn.metrics import confusion_matrix, classification_report

confusion_matrix(preds_binary, y_test)

array([[295,   2],
       [  0,  41]], dtype=int64)

In [13]:
print(classification_report(preds_binary, y_test))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00       297
           1       0.95      1.00      0.98        41

    accuracy                           0.99       338
   macro avg       0.98      1.00      0.99       338
weighted avg       0.99      0.99      0.99       338



# Conclusion

The model got good results - accuracy, recall, precision and f1-score of about 99%. Because of the good results and that the data includes pretty many different sizes and positions of leaves, I didn't use data augmentations on the dataset. If needed a more versatille model, some augmentations can be added like rotations, flippings and etc to handle more different leaf positions and sizes.